# Step 02: Group Rollup (Groups of Groups)

This notebook submits rollup grouping jobs to Moody's Risk Modeler for **groups that contain other groups**.

Rollup groups contain references to OTHER groups (not just analyses).

**Tasks:**
- Retrieve Grouping Rollup batch from Stage_01/Step_03
- Review rollup job configurations
- Submit rollup jobs to Moody's API
- Track job completion status

**Note:** If rollup groups reference child groups that don't exist yet, batch validation will detect this and block submission until the dependencies are resolved.

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch, validate_batch
from helpers.database import execute_query
from helpers.step import get_last_step_run
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType

# Flag to track validation state - allows notebook to complete gracefully on failure
validation_failed = False
validation_errors = []

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_02_Group_Rollup (chained).ipynb')

# Display context
ux.header("Group Rollup Batch (Groups of Groups)")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"Step tracking initialized for '{context.step_name}'")

## 2) Retrieve Grouping Rollup Batch

In [ ]:
# Retrieve Grouping Rollup batch from Stage_01/Step_03
ux.subheader("Retrieve Grouping Rollup Batch")

# Query for Stage_01/Step_03 step run to get batch IDs
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

# Check for GROUPING_ROLLUP batch
if BatchType.GROUPING_ROLLUP not in batches:
    ux.warning("No Grouping Rollup batch found in this cycle.")
    ux.info(f"Available batches: {list(batches.keys())}")
    ux.info("")
    ux.info("This cycle may not have any rollup groups (groups of groups).")
    ux.info("If you expected rollup groups, check your configuration file.")
    step.complete({'skipped': True, 'reason': 'No rollup batch in cycle'})
    ux.success("")
    ux.success("="*60)
    ux.success("STEP SKIPPED - No Grouping Rollup batch needed")
    ux.success("="*60)

    # Directly trigger next step (Grouping Summary) since there's no batch to chain from
    # Without this, the chain would break because chaining is batch-driven
    from helpers.step_chain import _build_notebook_path
    from helpers.notebook_executor import execute_next_step

    ux.info("")
    ux.info("Triggering next step (Grouping Summary) since no rollup batch to chain from...")

    try:
        next_notebook = _build_notebook_path(context.cycle_name, stage_num=5, step_num=3)
        result = execute_next_step(
            cycle_name=context.cycle_name,
            stage_num=5,
            step_num=3,
            notebook_path=next_notebook,
            timeout=3600
        )

        if result['success']:
            ux.success(f"Step 03 (Grouping Summary) executed successfully ({result['execution_time']:.1f}s)")
        else:
            ux.warning(f"Step 03 execution failed: {result['error'][:200]}")
    except Exception as e:
        ux.warning(f"Could not trigger Step 03: {str(e)}")

    # Set flag to skip remaining cells
    SKIP_REMAINING = True
else:
    SKIP_REMAINING = False
    rollup_batch_id = int(batches[BatchType.GROUPING_ROLLUP])

    ux.success(f"Retrieved Grouping Rollup batch: ID={rollup_batch_id}")
    step.log(f"Retrieved Grouping Rollup batch: ID={rollup_batch_id}")

## 3) Review Grouping Rollup Configuration

In [ ]:
# Verify batch status and display job information
if not SKIP_REMAINING:
    ux.subheader("Verify Batch Status")

    # Read batch details
    batch = read_batch(rollup_batch_id)

    batch_info = [
        ["Batch ID", batch['id']],
        ["Batch Type", batch['batch_type']],
        ["Status", batch['status']],
        ["Created", batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
    ]
    ux.table(batch_info, headers=["Property", "Value"])

    # Get jobs in batch
    jobs = get_batch_jobs(rollup_batch_id)
    job_count = len(jobs)

    ux.info(f"\nTotal rollup jobs: {job_count}")

    # Check entity existence for FINISHED/CANCELLED jobs to show what will be resubmitted
    from helpers.entity_validator import EntityValidator
    from helpers.constants import JobStatus
    from helpers.job import delete_groups_for_jobs

    validator = EntityValidator()
    irp_client = IRPClient()

    # Categorize jobs by status and entity existence
    pending_jobs = []      # INITIATED jobs - will be submitted
    skipped_jobs = []      # FINISHED/CANCELLED jobs where entity still exists - will be skipped
    resubmit_jobs = []     # FINISHED/CANCELLED jobs where entity is missing - will be resubmitted
    other_jobs = []        # Jobs in other states (SUBMITTED, RUNNING, etc.)

    for job in jobs:
        config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
        config_result = execute_query(config_query, (job['job_configuration_id'],))
        config_data = config_result.iloc[0]['job_configuration_data'] if not config_result.empty else {}
        group_name = config_data.get('Group_Name', 'N/A')
        items_count = len(config_data.get('items', []))
        display_name = f"{group_name} ({items_count} items)"

        if job['status'] == JobStatus.INITIATED:
            pending_jobs.append(display_name)
        elif job['status'] in (JobStatus.FINISHED, JobStatus.CANCELLED):
            entity_exists = validator.check_entity_exists_for_job(config_data, batch['batch_type'])
            if entity_exists:
                skipped_jobs.append(display_name)
            else:
                resubmit_jobs.append(display_name)
        else:
            other_jobs.append((display_name, job['status']))

    # Display categorized job summary
    ux.subheader("Job Summary")

    if pending_jobs:
        ux.info(f"\n Jobs to submit ({len(pending_jobs)}):")
        for name in pending_jobs[:10]:
            ux.info(f"  - {name}")
        if len(pending_jobs) > 10:
            ux.info(f"  ... and {len(pending_jobs) - 10} more")

    if resubmit_jobs:
        ux.warning(f"\n Jobs to resubmit - rollup group missing ({len(resubmit_jobs)}):")
        for name in resubmit_jobs[:10]:
            ux.warning(f"  - {name}")
        if len(resubmit_jobs) > 10:
            ux.warning(f"  ... and {len(resubmit_jobs) - 10} more")

    if skipped_jobs:
        ux.success(f"\n Jobs to skip - rollup group exists ({len(skipped_jobs)}):")
        for name in skipped_jobs[:10]:
            ux.success(f"  - {name}")
        if len(skipped_jobs) > 10:
            ux.success(f"  ... and {len(skipped_jobs) - 10} more")

    if other_jobs:
        ux.info(f"\n Jobs in progress ({len(other_jobs)}):")
        for name, status in other_jobs[:5]:
            ux.info(f"  - {name} ({status})")

    # Summary line
    total_to_process = len(pending_jobs) + len(resubmit_jobs)
    ux.info(f"\n Summary: {total_to_process} job(s) will be submitted, {len(skipped_jobs)} will be skipped")

    # Validate batch and get existing groups
    ux.subheader("Validate Batch")

    # Get job configurations for validation
    job_configs = []
    for job in jobs:
        config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
        config_result = execute_query(config_query, (job['job_configuration_id'],))
        if not config_result.empty:
            job_configs.append(config_result.iloc[0]['job_configuration_data'])

    # Validate using EntityValidator directly to get existing groups
    validation_messages, existing_rollup_groups = validator.validate_grouping_rollup_batch(job_configs)

    # Separate warnings from errors - warnings start with "WARN-"
    validation_warnings = [m for m in validation_messages if m.startswith("WARN-")]
    validation_errors_only = [m for m in validation_messages if not m.startswith("WARN-")]

    if validation_warnings:
        ux.warning("Validation warnings (submission will proceed):")
        for warning in validation_warnings:
            ux.warning(f"  {warning}")
        step.log(f"Validation warnings: {len(validation_warnings)}")

    # Handle existing rollup groups - auto-delete them
    groups_to_delete = []
    if existing_rollup_groups:
        ux.warning(f"\n Found {len(existing_rollup_groups)} existing rollup group(s) that will be deleted:")
        for group_name in existing_rollup_groups[:10]:
            ux.warning(f"  - {group_name}")
        if len(existing_rollup_groups) > 10:
            ux.warning(f"  ... and {len(existing_rollup_groups) - 10} more")

        # Build list for deletion
        groups_to_delete = [{'group_name': name} for name in existing_rollup_groups]

        ux.info("\n Deleting existing rollup groups...")
        delete_errors = delete_groups_for_jobs(groups_to_delete, irp_client)

        if delete_errors:
            ux.error(f"Failed to delete {len(delete_errors)} group(s):")
            for error in delete_errors:
                ux.error(f"  {error}")
            validation_failed = True
            validation_errors.extend([f"Failed to delete existing group: {e}" for e in delete_errors])
        else:
            ux.success(f"Successfully deleted {len(existing_rollup_groups)} existing rollup group(s)")
            step.log(f"Auto-deleted {len(existing_rollup_groups)} existing rollup groups")
            # Clear the group existence errors since we deleted them
            validation_errors_only = [e for e in validation_errors_only if "already exist" not in e.lower()]

    # Check for remaining validation errors
    if validation_errors_only:
        validation_failed = True
        ux.error("Batch validation failed:")
        for error in validation_errors_only:
            ux.error(f"  {error}")
        validation_errors.extend(validation_errors_only)
        step.log(f"Batch validation failed: {len(validation_errors_only)} error(s)")
    elif not validation_warnings and not existing_rollup_groups:
        ux.success("Batch validation passed")
        step.log(f"Verified batch: {total_to_process} jobs to submit, {len(skipped_jobs)} to skip")
    else:
        ux.success("Batch validation passed (with warnings/auto-deletions)")
        step.log(f"Verified batch: {total_to_process} jobs to submit, {len(skipped_jobs)} to skip")

## 4) Submit Grouping Rollup Batch to Moody's

In [ ]:
# Submit batch to Moody's API
if not SKIP_REMAINING:
    if validation_failed:
        ux.warning("⏭ Skipping submission due to validation failure")
        result = None
        failed_count = 0
    else:
        ux.subheader("Submit Rollup Batch to Moody's")

        ux.info("")
        ux.info("Submission Process:")
        ux.info("  - Each job will group the specified items (groups and/or analyses)")
        ux.info("  - Item names will be resolved to URIs")
        ux.info("  - Rollup results will be created in Moody's Risk Modeler")
        ux.info("  - Jobs will transition to SUBMITTED status")
        ux.info("  - Batch will transition to ACTIVE status")
        ux.info("")

        # Submit
        ux.info("\nSubmitting batch...")

        # Pass step.step_id to associate batch with this step (not the creation step)
        result = submit_batch(rollup_batch_id, IRPClient(), step_id=step.step_id)

        # Display results
        ux.success(f"\nBatch submission completed")
        ux.info(f"  Submitted: {result['submitted_jobs']} jobs")
        ux.info(f"  Status: {result['batch_status']}")

        # Check for errors
        failed_count = len([j for j in result['jobs'] if 'error' in j])
        if failed_count > 0:
            ux.warning(f"\n{failed_count} job(s) failed to submit")
            for job_result in result['jobs']:
                if 'error' in job_result:
                    ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

        step.log(f"Rollup batch submitted: {result['submitted_jobs']} jobs, {failed_count} failed")

## 5) Complete Step Execution

In [ ]:
# Complete step execution
if not SKIP_REMAINING:
    ux.header("Step Completion")

    if validation_failed:
        # Handle validation failure
        from helpers.step import update_step_run
        from helpers.constants import StepStatus
        from helpers.teams_notification import send_validation_failure_notification
        
        error_message = "\n".join(validation_errors)
        update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
        
        # Send Teams notification
        send_validation_failure_notification(
            cycle_name=context.cycle_name,
            stage_name=context.stage_name,
            step_name=context.step_name,
            validation_errors=validation_errors,
            notebook_path=str(context.notebook_path)
        )
        
        ux.error("\n" + "="*60)
        ux.error("BATCH VALIDATION FAILED")
        ux.error("="*60)
        ux.info(f"\nBatch ID: {rollup_batch_id}")
        ux.error(f"\nValidation errors ({len(validation_errors)}):")
        for error in validation_errors:
            ux.error(f"  {error}")
        ux.info("\nPlease fix the validation errors and retry.")

    elif failed_count > 0:
        # Handle submission failures
        failed_job_errors = [
            f"Job {j['job_id']}: {j['error']}" 
            for j in result['jobs'] if 'error' in j
        ]
        error_message = f"{failed_count} job(s) failed to submit:\n" + "\n".join(failed_job_errors)
        
        from helpers.step import update_step_run
        from helpers.constants import StepStatus
        update_step_run(step.run_id, StepStatus.FAILED, error_message=error_message)
        
        ux.error("\n" + "="*60)
        ux.error("BATCH SUBMISSION FAILED")
        ux.error("="*60)
        ux.info(f"\nBatch ID: {rollup_batch_id}")
        ux.info(f"Submitted: {result['submitted_jobs']} job(s)")
        ux.error(f"Failed: {failed_count} job(s)")
        ux.info("\nFailed jobs:")
        for error in failed_job_errors:
            ux.error(f"  {error}")
        ux.info("\nPlease review the errors and resubmit failed jobs.")

    else:
        # Complete the step successfully
        output_data = {
            'batch_id': rollup_batch_id,
            'batch_type': batch['batch_type'],
            'batch_status': result['batch_status'],
            'submitted_jobs': result['submitted_jobs'],
            'failed_jobs': failed_count
        }
        step.complete(output_data)

        ux.success("\n" + "="*60)
        ux.success("GROUPING ROLLUP BATCH SUBMITTED SUCCESSFULLY")
        ux.success("="*60)
        ux.info(f"\nSubmitted {result['submitted_jobs']} rollup job(s) to Moody's API")
        ux.info(f"Batch status: {result['batch_status']}")
        ux.info("\nNext: Monitor job progress or proceed to Export stage")